In [ ]:
import numpy as np
from glob import glob
import pandas as pd
import csv
from tsfresh import extract_features
from tsfresh.utilities.distribution import MultiprocessingDistributor

files = glob('data/*.csv')
totals = ['Total_PorosityQuantity', 'Total_PorosityQuality', 'Total_UnfilledZones', 'Total_FillingQuality', 'TOTAL_QUALITY']

df_list = list()
target = list()

for i, file in enumerate(files):
    df = pd.read_csv(file)

    if df.iloc[:,-1].unique()[0] == target_value:
        df['id'] = i
        target.append(df.TOTAL_QUALITY.unique()[0])
        df = df.drop(axis=1, columns=totals)   

        consta = [col for col in df if col.endswith(('K1', 'K2', 'K3'))]
        series = [col for col in df if col.endswith(('Time', 'id', 'Flow rate', 'Pressure'))]  

        distributor = MultiprocessingDistributor(n_workers=8, disable_progressbar=True, progressbar_title="Feature Extraction")
        extracted_features = extract_features(df[series], column_id='id', column_sort='Time', distributor=distributor)

        for j in consta:
            extracted_features[j] = df[j].unique()[0] 

        df_list.append(extracted_features)   



df = pd.concat(df_list)
df.head()        

In [3]:
from third_party.tseries_featurizer.base import TimeSeriesFeaturizer


ModuleNotFoundError: No module named 'third_party.tseries_featurizer.base'